Jadi ini untuk pipelineing preprocessing seluruh dataset dan juga akan disimpan hasil preprocessingnya

# Import depedencies

In [34]:
import pandas as pd
import re
import itertools
import string
import collections
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from IPython.utils import text
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
nltk.download('punkt')
import os


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tirtarumy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tirtarumy/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Load Data Set

In [27]:
# Daftar file path dataset
file_paths = ['/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/A1.csv',
                    '/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/A2.csv',
                    '/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/A3.csv',
                    '/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/P1.csv',
                    '/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/P2.csv',
                    '/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/P3.csv',
                    '/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/J1.csv',
                    '/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/J2.csv',
                    '/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/J3.csv',
                    '/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/C1.csv',
                    '/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/C2.csv',
                    '/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/C3.csv',
                    ]


# Preprocessing

In [28]:
def data_clean(indikator):
    indikator = indikator.lower()
    indikator = indikator.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    indikator = indikator.encode('ascii', 'replace').decode('ascii')
    indikator = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(#)", " ", indikator).split())
    indikator = re.sub(r"\d+", "", indikator)
    indikator = indikator.translate(str.maketrans("","",string.punctuation))
    indikator = indikator.strip()
    indikator = re.sub('\s+',' ',indikator)
    indikator = re.sub(r'\b\w\b', '', indikator)
    return indikator

In [29]:
normalized_word = pd.read_csv('/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/Research/Data/train_data/non_cum trainset/normalisasi1.csv')

normalized_word_dict = {}
for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]


/var/folders/s_/p4j9yxm93fv8_f_r0wk7t4kr0000gn/T/ipykernel_7339/3619471908.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalized_word_dict:
/var/folders/s_/p4j9yxm93fv8_f_r0wk7t4kr0000gn/T/ipykernel_7339/3619471908.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalized_word_dict[row[0]] = row[1]


In [30]:
def stopword_removal(indikator):
    filtering = stopwords.words('indonesian') + stopwords.words('english')
    data = [word for word in indikator if word.lower() not in filtering]
    return data
        

In [31]:
def stemming(indikator):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = [stemmer.stem(w) for w in indikator]
    d_clean = " ".join(do)
    return d_clean

In [32]:
def preprocess_file(file_path):
    # Baca file
    df = pd.read_csv(file_path)
    
    # Lakukan preprocessing pada kolom yang sesuai
    df['preprocessed_text'] = df['Indikator'].apply(data_clean)
    df['preprocessed_text'] = df['preprocessed_text'].apply(word_tokenize)
    df['preprocessed_text'] = df['preprocessed_text'].apply(normalized_term)
    df['preprocessed_text'] = df['preprocessed_text'].apply(stopword_removal)
    df['preprocessed_text'] = df['preprocessed_text'].apply(stemming)
    
    # Simpan dataframe ke file CSV dengan nama yang sesuai
    file_name = os.path.basename(file_path)
    output_file = f"{file_name.split('.')[0]}_preprocessing.csv"
    df.to_csv(output_file, index=False)
    print(f"File hasil preprocessing tersimpan sebagai: {output_file}")

In [35]:
# Loop untuk melakukan preprocessing pada setiap file
for file_path in file_paths:
    preprocess_file(file_path)

File hasil preprocessing tersimpan sebagai: A1_preprocessing.csv
File hasil preprocessing tersimpan sebagai: A2_preprocessing.csv
File hasil preprocessing tersimpan sebagai: A3_preprocessing.csv
File hasil preprocessing tersimpan sebagai: P1_preprocessing.csv
File hasil preprocessing tersimpan sebagai: P2_preprocessing.csv
File hasil preprocessing tersimpan sebagai: P3_preprocessing.csv
File hasil preprocessing tersimpan sebagai: J1_preprocessing.csv
File hasil preprocessing tersimpan sebagai: J2_preprocessing.csv
File hasil preprocessing tersimpan sebagai: J3_preprocessing.csv
File hasil preprocessing tersimpan sebagai: C1_preprocessing.csv
File hasil preprocessing tersimpan sebagai: C2_preprocessing.csv
File hasil preprocessing tersimpan sebagai: C3_preprocessing.csv


In [36]:
pd.read_csv('/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/A1_preprocessing.csv')

,Indikator,Tipe,preprocessed_text
0,melakukan pendekatan terhadap orang yang akan ...,E,dekat orang temu
1,Saya akan menerimanya dan mencoba untuk memper...,I,terima coba topik obrol
2,Tergantung penting gaknya,I,gantung
3,"tidak mau, tapi jika memang ingin bertemu ya d...",E,temu iya tempat ramai
4,"Bila diminta, saya akan berusaha sebisa mungki...",I,usaha bisa interaksi sopan orang
...,...,...,...
97,saya akan menerima dan mencoba beradaptasi,E,terima coba adaptasi
98,Dengan senang hati dan mencoba untuk berkenala...,E,senang hati coba nal tambah teman kerabat
99,menanyakan maksud dari pertemuan tersebut dan ...,I,maksud temu timbang
100,tidak mau karena tidak mengetahui identitas ap...,I,identitas apa


In [37]:
pd.read_csv('/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/C1_preprocessing.csv')

,Indikator,Tipe,preprocessed_text
0,"dari saya sendiri terdapat jadwal, namun saya ...",P,jadwal sesuai situasi
1,tergantung situasi,P,gantung situasi
2,Lebih fleksibel bergantung pada situasi,P,fleksibel gantung situasi
3,Cuman bilang ke temen² aku pen kesini blalala,P,cuman bilang temen pen kesini blalala
4,fleksibel bergantung pada situasi,P,fleksibel gantung situasi
...,...,...,...
87,tersusun rapi,J,susun rapi
88,Terjadwal agar tidak berantakan semua kegiatan...,J,jadwal beranta giat hari
89,saya lebih fleksibel bergantung pada situasi k...,P,fleksibel gantung situasi kecuali acara jadwal...
90,"membuat jadwal terlebih dahulu, jika ada sesua...",J,jadwal dadak gantung situasi


In [38]:
pd.read_csv('/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/P1_preprocessing.csv')

,Indikator,Tipe,preprocessed_text
0,mengajak orang tersebut berbicara hal-hal yang...,S,ajak orang bicara
1,Saya mengiyakan terlebih dahulu namun setelah ...,S,iya research kait kebenerannya
2,cara yang biasanya sering saya lakukan yaitu m...,S,laku bahasa tubuh marah murung dll pasti valid
3,menjadi pendengar yang baik,S,dengar
4,Saya hanya mendengarkan dan memperhatikan gera...,S,dengar perhati gerak gerik bahasa tubuh orang
...,...,...,...
97,dengan cara memahami dan mendengarkan orang be...,S,paham dengar orang bicara
98,Dengan mencari info akurat apakah info itu ben...,N,cari info akurat info kesalahpahaman
99,biasanya saya langsung diberitahu atau saya ya...,S,langsung diberitahu orang
100,"mendekatkan diri seolah-olah kita akrab, dan m...",S,dekat olah akrab cari informasi teman sosmed


In [39]:
pd.read_csv('/Users/tirtarumy/Documents/Data scientist porto/structured MBTI Classifiers/J1_preprocessing.csv')

,Indikator,Tipe,preprocessed_text
0,kalau saya sendiri lebih memilih menunggu tema...,F,pilih tunggu teman teman
1,menunggu,F,tunggu
2,Jika ujian saya akan menunggu teman2 yang lain...,F,uji tunggu teman selesai alas koreksi uji mana...
3,Mengulang ke soal no 1 smpe selesai selagi ada...,T,ulang smpe selesai selagi ecek
4,menunggu teman-teman yang lain selesai menjawa...,F,tunggu teman teman selesai
...,...,...,...
100,saya akan mengecek ulang hasil ulangan saya,T,ecek ulang hasil ulang
101,Menunggu teman-teman yang lain selesai dan men...,F,tunggu teman teman selesai ecek barangkali salah
102,"saya biasanya menunggu teman-teman yang lain, ...",F,tunggu teman teman orang selesai
103,sambil menunggu teman yang lainnya selesai men...,F,tunggu teman selesai koreksi salah
